In [1]:
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns 
from fcmpy.simulator.transfer import Sigmoid, Bivalent, Trivalent, HyperbolicTangent
from fcmpy import FcmSimulator, FcmIntervention 
import json
import csv
import os

In [4]:
# local imports
from run import run_simulation
from linearization import encode_diff, encode_state, input_format, diff_input_format

In [6]:
fp = open("sample_states.json")
states = json.load(fp)
# preparing the file
outfile = open('training-input.csv', 'w')
writer = csv.writer(outfile)
writer.writerow(('Prompt', 'Completion'))

def prepare(result):
    init_state, diff = result['init_state'], result['diff']
    initial = input_format(encode_state(init_state))
    difference = diff_input_format(encode_diff(diff))
    # completion = input("Write down a completion for above simulation:\nDo not write anything if you want to skip\n").strip()
    writer.writerow((f"Initial: {initial}\nDifference: {difference}",) )

for init_state in states:
    f, l = run_simulation(
        init_state
    ).values()

    first, last = f.to_dict('records')[0], l.to_dict('records')[0]
    # not keeping concepts that do not change
    diff_kv = {key: last[key]-first[key] for key in first if last[key] != first[key]}
    # sort concept values so that largest decrease appear first, largest increase appear last (primary concepts for dicussion)
    # along with central concept of FCM (which is obesity in our case)
    diff = sorted(diff_kv.items(), key=lambda x: x[1])

    prepare(dict(
        init_state=init_state,
        diff=diff,
    ))

outfile.close()
print("All simulation results written out to", outfile.name)


The values converged in the 7 state (e <= 0.001)
All simulation results written out to training-input.csv


In [3]:
init_state = {
    'Age': 0.6, 'Antidepressants': 1.0, 'Depression': 0.8, 'Belief in personal responsibility': 0.8, 'Exercise': 0.5, 'Fatness perceived as negative': 0.7,
    'Food intake': 0.4, 'Income': 0.8, 'Knowledge': 0.7, 'Obesity': 0.2, 'Physical health': 0.75, 'Stress': 0.9, 'Weight discrimination': 0.2
}
f, l = run_simulation(
    init_state
).values()

The values converged in the 7 state (e <= 0.001)


/Users/anish/opt/anaconda3/lib/python3.9/site-packages/fcmpy/simulator/simulator.py:64: UserWarning: When passing an initial state with a weight matrix type                 numpy.ndarray make sure that the order of the keys in the dictionary                 with the initial states matches the order of the column of the numpy.ndarray!
  warnings.warn("When passing an initial state with a weight matrix type \


In [4]:
f

,Age,Antidepressants,Depression,Belief in personal responsibility,Exercise,Fatness perceived as negative,Food intake,Income,Knowledge,Obesity,Physical health,Stress,Weight discrimination
0,0.6,1.0,0.8,0.8,0.5,0.7,0.4,0.8,0.7,0.2,0.75,0.9,0.2


In [5]:
l

,Age,Antidepressants,Depression,Belief in personal responsibility,Exercise,Fatness perceived as negative,Food intake,Income,Knowledge,Obesity,Physical health,Stress,Weight discrimination
6,0.6,0.763335,0.8,0.688211,0.787649,0.757695,0.80251,0.8,0.7,0.766019,0.4222,0.9,0.919228


### NOTE
Checking the difference between the last and first records as we are only interested in those. Mind it the difference is sorted, so that it is better to ONLY explain peculiar concepts (highest decreases/increases or very minor changes)

In [6]:
first, last = f.to_dict('records')[0], l.to_dict('records')[0]
diff_kv = {key: last[key]-first[key] for key in first if last[key] != first[key]}
# not keeping indifferent keys
diff = sorted(diff_kv.items(), key=lambda x: x[1])
print('Difference', diff)


Difference [('Physical health', -0.3277996028071101), ('Antidepressants', -0.23666502335745), ('Belief in personal responsibility', -0.11178931129657799), ('Fatness perceived as negative', 0.05769529884131608), ('Exercise', 0.28764949582051835), ('Food intake', 0.4025102826623449), ('Obesity', 0.56601885890069), ('Weight discrimination', 0.719227966320684)]


### Preview the state and difference of the LAST result

In [7]:
from linearization import encode_diff, encode_state, input_format, diff_input_format

print(init_state)
print('-'*10, 'Difference', '-'*10)
print(diff)
print(encode_diff(diff))
print('\nPart of the training data, this is written out to the file\n')
print('Initial: ' + input_format(encode_state(init_state)))
print('Difference: ' + diff_input_format(encode_diff(diff)))


{'Age': 0.6, 'Antidepressants': 1.0, 'Depression': 0.8, 'Belief in personal responsibility': 0.8, 'Exercise': 0.5, 'Fatness perceived as negative': 0.7, 'Food intake': 0.4, 'Income': 0.8, 'Knowledge': 0.7, 'Obesity': 0.2, 'Physical health': 0.75, 'Stress': 0.9, 'Weight discrimination': 0.2}
---------- Difference ----------
[('Physical health', -0.3277996028071101), ('Antidepressants', -0.23666502335745), ('Belief in personal responsibility', -0.11178931129657799), ('Fatness perceived as negative', 0.05769529884131608), ('Exercise', 0.28764949582051835), ('Food intake', 0.4025102826623449), ('Obesity', 0.56601885890069), ('Weight discrimination', 0.719227966320684)]
[('Physical health', 'decrease', 'small'), ('Antidepressants', 'decrease', 'small'), ('Belief in personal responsibility', 'decrease', 'very low'), ('Fatness perceived as negative', 'increase', 'negligible'), ('Exercise', 'increase', 'small'), ('Food intake', 'grow', 'average'), ('Obesity', 'increase', 'average'), ('Weight d

### Linearized Representation
#### Two different entities are there: Initial and Difference

```
Initial: <S> Age <V> seniors <E><S> Antidepressants <V> very low <E><S> Depression <V> low <E><S> Belief in personal responsibility <V> average <E><S> Exercise <V> very high <E><S> Fatness perceived as negative <V> average <E><S> Food intake <V> noticeably high <E><S> Income <V> very low <E><S> Knowledge <V> high <E><S> Obesity <V> very high <E><S> Physical health <V> average <E><S> Stress <V> average <E><S> Weight discrimination <V> extremely high <E>
Difference: <S> Exercise <C> decrease <V> very small <E><S> Obesity <C> reduce <V> noticeably low <E><S> Food intake <C> decrease <V> very low <E><S> Weight discrimination <C> grow <V> very small <E><S> Physical health <C> grow <V> very low <E><S> Belief in personal responsibility <C> grow <V> very low <E><S> Fatness perceived as negative <C> grow <V> low <E><S> Antidepressants <C> increase <V> elevated <E>
```

### Print the states and their differences for the simulations performed above

In [8]:
import csv
import os

outfile = open('training-input.csv', 'w')
writer = csv.writer(outfile)
writer.writerow(('Prompt', 'Completion'))

def prepare(result):
    init_state, diff = result['init_state'], result['diff']
    initial = input_format(encode_state(init_state))
    difference = diff_input_format(encode_diff(diff))
    # completion = input("Write down a completion for above simulation:\nDo not write anything if you want to skip\n").strip()
    writer.writerow((f"Initial: {initial}\nDifference: {difference}",) )

prepare(dict(
    init_state=init_state,
    diff=diff,
))

outfile.close()
print("All simulation results written out to", outfile.name)

All simulation results written out to  training-input.csv


In [10]:
'''
part = res.loc[:, ~res.columns.isin(['Age', 'Income'])]
part.plot(kind='line', colormap='Greens')
'''